# Google Colab MCP Server Setup

This notebook sets up a Jupyter Server in Google Colab that can be accessed remotely via Cloudflare tunnel.

## Architecture

```
Your Machine:                          Google Colab:
┌─────────────────┐                   ┌──────────────────┐
│ Claude Desktop  │                   │ Jupyter Server   │
│       ↓         │                   │        ↓         │
│  MCP Server     │                   │  Python Kernel   │
│       ↓         │                   │   (GPU/TPU)      │
│  JupyterLab  ───┼──→ Cloudflare ───→│                  │
│   (Browser)     │      Tunnel       │                  │
└─────────────────┘                   └──────────────────┘
```

## What This Enables

- ✅ Real-time notebook collaboration with AI agents
- ✅ Use Colab's GPU/TPU for ML experiments
- ✅ Persistent access via your subdomain
- ✅ Works with existing Jupyter MCP Server

## Prerequisites

1. Cloudflare account
2. Domain configured in Cloudflare
3. Google Colab Pro (recommended for persistent sessions)

## Setup Steps

Run the cells below in order.

## Step 1: Clone Setup Repository

In [ ]:
# Clone the MCP Colab repository (or download scripts manually)
!git clone https://github.com/YOUR_USERNAME/mcp-colab.git /content/mcp-colab
%cd /content/mcp-colab/colab

## Step 2: Run Setup Script

This installs Jupyter Server, extensions, and cloudflared.

In [ ]:
# Make scripts executable
!chmod +x *.sh

# Run setup
!./setup_jupyter_server.sh

## Step 3: Configure Cloudflare Tunnel

### Option A: First-Time Setup

Run these commands in the **Colab terminal** (not in a notebook cell):

```bash
# 1. Login to Cloudflare (opens browser for auth)
cloudflared tunnel login

# 2. Create a tunnel
cloudflared tunnel create colab-tunnel

# 3. Note the tunnel ID shown in output
# It will create a credentials file at:
# ~/.cloudflared/<TUNNEL_ID>.json

# 4. Move credentials to /content for persistence
mkdir -p /content/tunnel-credentials
cp ~/.cloudflared/*.json /content/tunnel-credentials/

# 5. Create tunnel config
# Copy template and edit with your tunnel ID
cp cloudflare-tunnel.yml.template /content/cloudflare-tunnel.yml
nano /content/cloudflare-tunnel.yml  # Edit YOUR_TUNNEL_ID_HERE

# 6. Configure DNS in Cloudflare dashboard
# Add CNAME record:
#   Name: colab.leonard-schneider.com
#   Target: <TUNNEL_ID>.cfargotunnel.com
```

### Option B: Restore from Drive

If you've set this up before and saved credentials to Google Drive:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy credentials from Drive
!mkdir -p /content/tunnel-credentials
!cp /content/drive/MyDrive/colab-tunnel/*.json /content/tunnel-credentials/
!cp /content/drive/MyDrive/colab-tunnel/cloudflare-tunnel.yml /content/

print("✓ Credentials restored from Google Drive")

## Step 4: Start Jupyter Server

In [ ]:
!./start_jupyter_server.sh

## Step 5: Start Cloudflare Tunnel

In [ ]:
!./start_cloudflare_tunnel.sh

## Step 6: Verify Setup

In [ ]:
import requests
import time

print("Checking services...\n")

# Check Jupyter Server (local)
try:
    response = requests.get("http://localhost:8888/api", timeout=5)
    if response.status_code == 200:
        print("✓ Jupyter Server is running locally")
    else:
        print("✗ Jupyter Server returned status:", response.status_code)
except Exception as e:
    print("✗ Jupyter Server not accessible:", e)

# Check tunnel (remote)
time.sleep(2)  # Give tunnel a moment to establish
try:
    response = requests.get("https://colab.leonard-schneider.com/api", timeout=10)
    if response.status_code == 200:
        print("✓ Cloudflare Tunnel is working")
        print("\n🎉 Setup complete! Connect JupyterLab to:")
        print("   https://colab.leonard-schneider.com")
    else:
        print("✗ Tunnel returned status:", response.status_code)
except Exception as e:
    print("✗ Tunnel not accessible:", e)
    print("\nTroubleshooting:")
    print("1. Check tunnel logs: !tail /tmp/cloudflared.log")
    print("2. Verify DNS is configured in Cloudflare")
    print("3. Wait a few minutes for DNS propagation")

## Step 7: Connect Your Local JupyterLab

On your local machine, add this server to JupyterLab:

1. Open JupyterLab locally
2. File → Hub Control Panel (or File → New Launcher)
3. Click "New Server" or "Add Server"
4. Enter URL: `https://colab.leonard-schneider.com`
5. No token needed (configured without authentication)

**Security Note**: The server has no token authentication because Cloudflare tunnel handles security. Make sure your Cloudflare settings restrict access appropriately!

## Monitoring & Maintenance

### View Logs

In [ ]:
# Jupyter Server logs
!tail -n 50 /tmp/jupyter-server.log

In [ ]:
# Cloudflare Tunnel logs
!tail -n 50 /tmp/cloudflared.log

### Check Running Processes

In [ ]:
!ps aux | grep -E 'jupyter|cloudflared' | grep -v grep

### Stop Services

In [ ]:
# Stop Jupyter Server
!pkill -f 'jupyter-lab'

# Stop Cloudflare Tunnel
!pkill -f 'cloudflared tunnel'

print("Services stopped")

## Save Credentials to Drive

Save your tunnel credentials to Google Drive so you don't have to reconfigure each session:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create backup directory
!mkdir -p /content/drive/MyDrive/colab-tunnel

# Copy credentials and config
!cp -r /content/tunnel-credentials/* /content/drive/MyDrive/colab-tunnel/
!cp /content/cloudflare-tunnel.yml /content/drive/MyDrive/colab-tunnel/

print("✓ Credentials saved to Google Drive")
print("  Location: /content/drive/MyDrive/colab-tunnel/")